In [1]:
#%pip install oauth2client

In [2]:
import os
import asyncio
import time
from pathlib import Path
from types import SimpleNamespace
from typing import Optional, List
from dataclasses import dataclass, field

import header
from header import __root__

# External modules
from src import gs
from src.suppliers.suppliers_list import *
from src.suppliers.get_graber_by_supplier  import get_graber_by_supplier_prefix, get_graber_by_supplier_url
from src.webdriver.driver import Driver
from src.webdriver.chrome import Chrome
from src.webdriver.firefox import Firefox
from src.llm.gemini import GoogleGenerativeAi
from src.llm.openai.model import OpenAIModel
from src.endpoints.prestashop.product import PrestaProduct
from src.endpoints.prestashop.language import PrestaLanguage
from src.endpoints.prestashop.product_fields import ProductFields
from src.endpoints.advertisement.facebook.scenarios.post_message import (
    post_message,
)
from src.utils.file import read_text_file, save_text_file, get_filenames_from_directory

from src.utils.jjson import j_loads, j_loads_ns, j_dumps
from src.utils.image import get_image_bytes, get_raw_image_data
from src.logger.logger import logger

2025-04-20 21:34:18,551 - ERROR - ❌ Error fetching data: 404 
NoneType: None
2025-04-20 21:34:22,979 - INFO - Anonymized telemetry enabled. See https://docs.browser-use.com/development/telemetry for more information.


In [3]:
#d = Driver(Firefox, window_mode = 'headless')
d = Driver(Firefox)

2025-04-20 21:34:23,209 - INFO - ℹ️ Starting Firefox WebDriver 
2025-04-20 21:34:27,811 - INFO - ℹ️ Browser started successfully, window_mode='headless' 


In [4]:
scenario_hb:dict = {
    "supplier_prefix":"hb",
   "url": "https://hbdeadsea.co.il/product-category/aromatherapy/diffusers/",
    "train_product_url":"https://hbdeadsea.co.il/product/%d7%a7%d7%a8%d7%9d-%d7%a8%d7%92%d7%9c%d7%99%d7%99%d7%9d-%d7%90%d7%99%d7%a0%d7%98%d7%a0%d7%a1%d7%99%d7%91%d7%99-%d7%9e%d7%95%d7%a2%d7%a9%d7%a8-%d7%91%d7%91%d7%95%d7%a5-%d7%9e%d7%99%d7%9d-%d7%94%d7%9e-2/",
   "name": "מפיצי ריח ומילוי",
   "condition": "new",
   "presta_categories": {
     "default_category": 11246,
     "additional_categories": []
   }
}
scenario_etzmaleh:dict = {
    "supplier_prefix":"etzmaleh",
    "url":"https://www.etzmaleh.co.il/montessori_collection",
    "train_product_url":"https://www.etzmaleh.co.il/montessori_transition_bed",
    "name":"הקולקציה המונטסורית",
    "condition":"new",
       "presta_categories": {
     "default_category": 11390,
     "additional_categories": [11389]
   }
}
scenario_amazon:dict =  {
    "supplier_prefix":"amazon",
      "url": "https://amzn.to/3OhRz2g",
        "train_product_url":"https://www.amazon.com/C%C3%A1-dOro-Hippie-Colored-Murano-Style/dp/B09N53XSQB/ref=sr_1_2_sspa?crid=24Q0ZZYVNOQMP&dib=eyJ2IjoiMSJ9.WhN_5Deyh2Yz9gRyrG1anDCr1UB8tnpHH_pJePpURaaciFWg_5Ft0XuS-e3A67g3rO1RECMmgpmRLEJOI7Zj6JyAd9DJh6dLSwUcHgHnJiKhmBmpa-AlDotmiq0-4Q_b90WkEmSsgIzC4L1Yc_KssvHa7bj6LGl6fMM4VuctS4nDQ3vYigYzB3jzP9Q3nT5BRv5_yRigaF-hDpPc0lL5wvo5gQ4i-9Jcy4vHyHNX04ZpoQBXrwcwLo3XWoruWSmnrgvQ5XxRzLntClGxBSN3cTjsunsHYzzKSp_VYx631JM.U1bRX0eDJ5c13nETwkLbN9BPt6-OtTKUvvRQiGodOd0&dib_tag=se&keywords=Art+Deco+murano+glass&linkCode=sl2&linkId=1a5da5b6a02f09a4d8fe47362e06cf3a&qid=1745171225&sprefix=art+deco+murano+glass%2Caps%2C230&sr=8-2-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1",
      "condition": "new",
      "presta_categories": {
        "default_category": 11209,
        "additional_categories": [ 11206, 11388 ]
      }
    }
scenario_aliexpress:dict =  {
    "supplier_prefix":"aliexpress",
      "url": "https://www.aliexpress.com",
    "train_product_url":"https://www.aliexpress.com/item/1005008108060545.html?spm=a2g0o.tm1000020483.d1.1.28f9474cPkErMV&sourceType=561&pvid=1a630f56-af44-4a5e-8fa6-ba39f7382734&pdp_ext_f=%7B%22ship_from%22:%22CN%22,%22sku_id%22:%2212000043799911402%22%7D&scm=1007.28480.422277.0&scm-url=1007.28480.422277.0&scm_id=1007.28480.422277.0&pdp_npi=4%40dis%21ILS%21%E2%82%AA%20119.48%21%E2%82%AA%2058.54%21%21%21231.15%21113.26%21%402141131717451712898942519ec7de%2112000043799911402%21gsd%21IL%213690978535%21&channel=sd&aecmd=true",
      "condition": "new",
      "presta_categories": {
        "default_category": 4292,
        "additional_categories": []
      }
    }


In [5]:
scenarios_list:list = [scenario_hb,scenario_etzmaleh,scenario_amazon,scenario_aliexpress]

In [6]:
async def process_scenarios(scenarios_list:List[dict]) -> bool:
    """Исполняет сценарии для сбора товаров от поставщиков"""
    for scenario in scenarios_list:
        graber:Graber = get_graber_by_supplier_prefix(d, scenario['supplier_prefix'], lang_index = 2)
        d.get_url(scenario['train_product_url'])
        res = await graber.grab_page_async()

In [7]:
await process_scenarios(scenarios_list)

2025-04-20 21:35:15,527 - DEBUG - 🐛 Empty locator provided. 
2025-04-20 21:35:15,527 - ERROR - ❌ SKU not found!  
2025-04-20 21:35:15,639 - DEBUG - 🐛 Empty locator provided. 
2025-04-20 21:35:15,639 - DEBUG - 🐛 Empty locator provided. 


UnboundLocalError: cannot access local variable 'grabber' where it is not associated with a value